In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# !git pull

In [3]:
import tensorflow as tf
from glob import glob
import malaya_speech.config
import malaya_speech.train.model.quartznet as quartznet
import malaya_speech.train.model.ctc as ctc
import malaya_speech.train as train
import numpy as np


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(config)
n_mels = config['num_feature_bins']
initial_learning_rate = 1e-3
max_gradient_norm = 5.0
train_steps = 1000

In [5]:
def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(mel_fbanks)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = mel_fbanks
    example['inputs_length'] = length

    return example

In [6]:
def parse(serialized_example):

    data_fields = {
        'waveforms': tf.VarLenFeature(tf.float32),
        'targets': tf.VarLenFeature(tf.int64),
    }
    features = tf.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['inputs', 'inputs_length', 'targets']:
            features.pop(k, None)

    return features


def get_dataset(path, batch_size = 32, shuffle_size = 32, thread_count = 24):
    def get():
        files = glob(path)
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'inputs_length': tf.constant(0, dtype = tf.int32),
                'targets': tf.constant(0, dtype = tf.int64),
            },
        )
        dataset = dataset.repeat()
        return dataset

    return get

In [7]:
def model_fn(features, labels, mode, params):

    model = quartznet.Model(
        features['inputs'], features['inputs_length'][:, 0], mode = 'train'
    )
    logits = tf.layers.dense(
        model.logits['outputs'], malaya_speech.char.VOCAB_SIZE
    )
    seq_lens = model.logits['src_length']

    targets_int32 = tf.cast(features['targets'], tf.int32)

    mean_error, sum_error, sum_weight = ctc.loss.ctc_loss(
        logits, targets_int32, seq_lens
    )

    loss = mean_error
    accuracy = ctc.metrics.ctc_sequence_accuracy(
        logits, targets_int32, seq_lens
    )

    tf.identity(loss, 'train_loss')
    tf.identity(accuracy, name = 'train_accuracy')

    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()
        learning_rate = tf.train.polynomial_decay(
            initial_learning_rate,
            global_step,
            train_steps,
            end_learning_rate = 1e-4,
            power = 1.0,
            cycle = False,
        )
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        train_op = optimizer.minimize(loss, global_step = global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
            mode = mode, loss = loss, train_op = train_op
        )

    elif mode == tf.estimator.ModeKeys.EVAL:

        estimator_spec = tf.estimator.EstimatorSpec(
            mode = tf.estimator.ModeKeys.EVAL,
            loss = loss,
            eval_metric_ops = {
                'accuracy': ctc.metrics.ctc_sequence_accuracy_estimator(
                    logits, targets_int32, seq_lens
                ),
                'WER': ctc.metrics.word_error_rate_estimator(
                    logits, targets_int32
                ),
            },
        )

    return estimator_spec

In [8]:
train_hooks = [
    tf.train.LoggingTensorHook(
        ['train_accuracy', 'train_loss'], every_n_iter = 1
    )
]
train_dataset = get_dataset('tolong-sebut/data/tolong-sebut-train*')
dev_dataset = get_dataset('tolong-sebut/data/tolong-sebut-dev*')

In [9]:
!rm -rf asr-quartznet

In [10]:
train.run_training(
    train_fn = train_dataset,
    model_fn = model_fn,
    model_dir = 'asr-quartznet',
    num_gpus = 1,
    log_step = 1,
    save_checkpoint_step = 200,
    max_steps = train_steps,
    eval_fn = dev_dataset,
    train_hooks = train_hooks,
)



INFO:tensorflow:Using config: {'_model_dir': 'asr-quartznet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f13f003ef98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation

INFO:tensorflow:loss = 69.670364, step = 13 (0.625 sec)
INFO:tensorflow:global_step/sec: 1.59812
INFO:tensorflow:train_accuracy = 0.016927084, train_loss = 74.92718 (0.626 sec)
INFO:tensorflow:loss = 74.92718, step = 14 (0.626 sec)
INFO:tensorflow:global_step/sec: 1.54822
INFO:tensorflow:train_accuracy = 0.007989348, train_loss = 67.74375 (0.646 sec)
INFO:tensorflow:loss = 67.74375, step = 15 (0.647 sec)
INFO:tensorflow:global_step/sec: 1.57157
INFO:tensorflow:train_accuracy = 0.03192848, train_loss = 77.726746 (0.636 sec)
INFO:tensorflow:loss = 77.726746, step = 16 (0.636 sec)
INFO:tensorflow:global_step/sec: 1.7516
INFO:tensorflow:train_accuracy = 0.031128405, train_loss = 77.6071 (0.571 sec)
INFO:tensorflow:loss = 77.6071, step = 17 (0.572 sec)
INFO:tensorflow:global_step/sec: 1.69397
INFO:tensorflow:train_accuracy = 0.0078023407, train_loss = 68.87024 (0.590 sec)
INFO:tensorflow:loss = 68.87024, step = 18 (0.590 sec)
INFO:tensorflow:global_step/sec: 1.55234
INFO:tensorflow:train_ac

INFO:tensorflow:loss = 54.689804, step = 60 (0.496 sec)
INFO:tensorflow:global_step/sec: 1.822
INFO:tensorflow:train_accuracy = 0.05324675, train_loss = 56.476532 (0.549 sec)
INFO:tensorflow:loss = 56.476532, step = 61 (0.549 sec)
INFO:tensorflow:global_step/sec: 1.93062
INFO:tensorflow:train_accuracy = 0.037958115, train_loss = 59.659557 (0.518 sec)
INFO:tensorflow:loss = 59.659557, step = 62 (0.518 sec)
INFO:tensorflow:global_step/sec: 1.87153
INFO:tensorflow:train_accuracy = 0.04654088, train_loss = 59.812447 (0.534 sec)
INFO:tensorflow:loss = 59.812447, step = 63 (0.534 sec)
INFO:tensorflow:global_step/sec: 1.89226
INFO:tensorflow:train_accuracy = 0.032258064, train_loss = 51.74657 (0.529 sec)
INFO:tensorflow:loss = 51.74657, step = 64 (0.529 sec)
INFO:tensorflow:global_step/sec: 1.81604
INFO:tensorflow:train_accuracy = 0.041612484, train_loss = 59.7442 (0.551 sec)
INFO:tensorflow:loss = 59.7442, step = 65 (0.551 sec)
INFO:tensorflow:global_step/sec: 1.62705
INFO:tensorflow:train_a

INFO:tensorflow:train_accuracy = 0.12840466, train_loss = 49.470116 (0.545 sec)
INFO:tensorflow:loss = 49.470116, step = 107 (0.545 sec)
INFO:tensorflow:global_step/sec: 1.91397
INFO:tensorflow:train_accuracy = 0.16821192, train_loss = 44.4474 (0.523 sec)
INFO:tensorflow:loss = 44.4474, step = 108 (0.523 sec)
INFO:tensorflow:global_step/sec: 1.88424
INFO:tensorflow:train_accuracy = 0.12549537, train_loss = 41.46634 (0.531 sec)
INFO:tensorflow:loss = 41.46634, step = 109 (0.531 sec)
INFO:tensorflow:global_step/sec: 1.82204
INFO:tensorflow:train_accuracy = 0.14138818, train_loss = 49.29923 (0.549 sec)
INFO:tensorflow:loss = 49.29923, step = 110 (0.549 sec)
INFO:tensorflow:global_step/sec: 1.8114
INFO:tensorflow:train_accuracy = 0.12827225, train_loss = 45.673637 (0.552 sec)
INFO:tensorflow:loss = 45.673637, step = 111 (0.552 sec)
INFO:tensorflow:global_step/sec: 1.83569
INFO:tensorflow:train_accuracy = 0.10981912, train_loss = 44.434998 (0.545 sec)
INFO:tensorflow:loss = 44.434998, step 

INFO:tensorflow:loss = 33.46064, step = 153 (0.503 sec)
INFO:tensorflow:global_step/sec: 1.98867
INFO:tensorflow:train_accuracy = 0.31530344, train_loss = 31.209377 (0.503 sec)
INFO:tensorflow:loss = 31.209377, step = 154 (0.503 sec)
INFO:tensorflow:global_step/sec: 1.728
INFO:tensorflow:train_accuracy = 0.26441103, train_loss = 36.82923 (0.579 sec)
INFO:tensorflow:loss = 36.82923, step = 155 (0.579 sec)
INFO:tensorflow:global_step/sec: 1.82308
INFO:tensorflow:train_accuracy = 0.314888, train_loss = 30.534271 (0.548 sec)
INFO:tensorflow:loss = 30.534271, step = 156 (0.548 sec)
INFO:tensorflow:global_step/sec: 1.83945
INFO:tensorflow:train_accuracy = 0.38, train_loss = 29.77198 (0.544 sec)
INFO:tensorflow:loss = 29.77198, step = 157 (0.544 sec)
INFO:tensorflow:global_step/sec: 1.63486
INFO:tensorflow:train_accuracy = 0.33375958, train_loss = 34.230366 (0.612 sec)
INFO:tensorflow:loss = 34.230366, step = 158 (0.611 sec)
INFO:tensorflow:global_step/sec: 1.88538
INFO:tensorflow:train_accur

...

INFO:tensorflow:loss = 25.450104, step = 800 (0.616 sec)
INFO:tensorflow:global_step/sec: 1.82734
INFO:tensorflow:train_accuracy = 0.5013089, train_loss = 25.118227 (0.546 sec)
INFO:tensorflow:loss = 25.118227, step = 801 (0.545 sec)
INFO:tensorflow:global_step/sec: 1.82003
INFO:tensorflow:train_accuracy = 0.5246114, train_loss = 22.718306 (0.549 sec)
INFO:tensorflow:loss = 22.718306, step = 802 (0.549 sec)
INFO:tensorflow:global_step/sec: 1.61895
INFO:tensorflow:train_accuracy = 0.4866242, train_loss = 26.246677 (0.618 sec)
INFO:tensorflow:loss = 26.246677, step = 803 (0.618 sec)
INFO:tensorflow:global_step/sec: 1.75165
INFO:tensorflow:train_accuracy = 0.5294118, train_loss = 23.632244 (0.571 sec)
INFO:tensorflow:loss = 23.632244, step = 804 (0.571 sec)
INFO:tensorflow:global_step/sec: 1.86634
INFO:tensorflow:train_accuracy = 0.51915455, train_loss = 25.2184 (0.535 sec)
INFO:tensorflow:loss = 25.2184, step = 805 (0.535 sec)
INFO:tensorflow:global_step/sec: 1.95629
INFO:tensorflow:trai

INFO:tensorflow:train_accuracy = 0.5349144, train_loss = 22.740362 (0.531 sec)
INFO:tensorflow:loss = 22.740362, step = 847 (0.531 sec)
INFO:tensorflow:global_step/sec: 1.61571
INFO:tensorflow:train_accuracy = 0.50318474, train_loss = 26.456608 (0.619 sec)
INFO:tensorflow:loss = 26.456608, step = 848 (0.619 sec)
INFO:tensorflow:global_step/sec: 1.84199
INFO:tensorflow:train_accuracy = 0.5191816, train_loss = 24.333862 (0.543 sec)
INFO:tensorflow:loss = 24.333862, step = 849 (0.543 sec)
INFO:tensorflow:global_step/sec: 1.77186
INFO:tensorflow:train_accuracy = 0.5089744, train_loss = 23.870316 (0.564 sec)
INFO:tensorflow:loss = 23.870316, step = 850 (0.564 sec)
INFO:tensorflow:global_step/sec: 2.01192
INFO:tensorflow:train_accuracy = 0.5418327, train_loss = 24.544678 (0.497 sec)
INFO:tensorflow:loss = 24.544678, step = 851 (0.497 sec)
INFO:tensorflow:global_step/sec: 1.8588
INFO:tensorflow:train_accuracy = 0.51846963, train_loss = 24.528423 (0.538 sec)
INFO:tensorflow:loss = 24.528423, s

INFO:tensorflow:loss = 29.726278, step = 893 (0.608 sec)
INFO:tensorflow:global_step/sec: 1.64013
INFO:tensorflow:train_accuracy = 0.496134, train_loss = 29.29231 (0.610 sec)
INFO:tensorflow:loss = 29.29231, step = 894 (0.610 sec)
INFO:tensorflow:global_step/sec: 1.8499
INFO:tensorflow:train_accuracy = 0.49167734, train_loss = 28.575726 (0.541 sec)
INFO:tensorflow:loss = 28.575726, step = 895 (0.541 sec)
INFO:tensorflow:global_step/sec: 1.77797
INFO:tensorflow:train_accuracy = 0.47493404, train_loss = 28.053143 (0.562 sec)
INFO:tensorflow:loss = 28.053143, step = 896 (0.562 sec)
INFO:tensorflow:global_step/sec: 1.86205
INFO:tensorflow:train_accuracy = 0.48416886, train_loss = 27.203428 (0.537 sec)
INFO:tensorflow:loss = 27.203428, step = 897 (0.538 sec)
INFO:tensorflow:global_step/sec: 1.98181
INFO:tensorflow:train_accuracy = 0.49271524, train_loss = 26.616747 (0.504 sec)
INFO:tensorflow:loss = 26.616747, step = 898 (0.504 sec)
INFO:tensorflow:global_step/sec: 1.81731
INFO:tensorflow:t

INFO:tensorflow:global_step/sec: 1.76731
INFO:tensorflow:train_accuracy = 0.49159122, train_loss = 26.735216 (0.566 sec)
INFO:tensorflow:loss = 26.735216, step = 940 (0.565 sec)
INFO:tensorflow:global_step/sec: 1.63913
INFO:tensorflow:train_accuracy = 0.48358586, train_loss = 28.95419 (0.610 sec)
INFO:tensorflow:loss = 28.95419, step = 941 (0.611 sec)
INFO:tensorflow:global_step/sec: 1.79294
INFO:tensorflow:train_accuracy = 0.48677248, train_loss = 25.70279 (0.558 sec)
INFO:tensorflow:loss = 25.70279, step = 942 (0.557 sec)
INFO:tensorflow:global_step/sec: 1.86671
INFO:tensorflow:train_accuracy = 0.5185185, train_loss = 24.408476 (0.536 sec)
INFO:tensorflow:loss = 24.408476, step = 943 (0.536 sec)
INFO:tensorflow:global_step/sec: 1.86003
INFO:tensorflow:train_accuracy = 0.5109114, train_loss = 26.476772 (0.537 sec)
INFO:tensorflow:loss = 26.476772, step = 944 (0.537 sec)
INFO:tensorflow:global_step/sec: 1.73968
INFO:tensorflow:train_accuracy = 0.52476573, train_loss = 25.745953 (0.575 

INFO:tensorflow:loss = 26.304844, step = 986 (0.531 sec)
INFO:tensorflow:global_step/sec: 1.6338
INFO:tensorflow:train_accuracy = 0.51225805, train_loss = 25.26618 (0.612 sec)
INFO:tensorflow:loss = 25.26618, step = 987 (0.613 sec)
INFO:tensorflow:global_step/sec: 1.76678
INFO:tensorflow:train_accuracy = 0.5228162, train_loss = 22.909328 (0.566 sec)
INFO:tensorflow:loss = 22.909328, step = 988 (0.566 sec)
INFO:tensorflow:global_step/sec: 1.64154
INFO:tensorflow:train_accuracy = 0.5181818, train_loss = 25.061558 (0.609 sec)
INFO:tensorflow:loss = 25.061558, step = 989 (0.609 sec)
INFO:tensorflow:global_step/sec: 1.6837
INFO:tensorflow:train_accuracy = 0.5411765, train_loss = 25.154573 (0.594 sec)
INFO:tensorflow:loss = 25.154573, step = 990 (0.594 sec)
INFO:tensorflow:global_step/sec: 1.8817
INFO:tensorflow:train_accuracy = 0.5498008, train_loss = 22.334309 (0.532 sec)
INFO:tensorflow:loss = 22.334309, step = 991 (0.532 sec)
INFO:tensorflow:global_step/sec: 1.71527
INFO:tensorflow:train